In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re #regular expressions
import spacy #natural language processing import pandas as pd
from nltk import tokenize # fast sentence tokenization
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.stem import WordNetLemmatizer
from collections import Counter
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Making Product Count_Final Project/Diabetes Continuous Glucose Monitoring – Data Export.xlsx')
df.head()

,Post ID,Sound Bite Text,Ratings and Scores,Title,Source Type,Post Type,Is Paid,Media Type,URL,Media Link,...,LexisNexis Company - Any,LexisNexis Person - High,LexisNexis Person - Any,LexisNexis Institution - High,LexisNexis Institution - Any,LexisNexis Subject Group 1,LexisNexis Subject 1,LexisNexis Subject Group 2,LexisNexis Subject 2,LexisNexis Other Subjects
0,BRDRDT2-t1_imq98sr,My numbers are great now. Estimated a1c of 7%i...,NaN,Have you been denied a second/third pump? Feel...,Forums,Replies and Comments,No,No Media,https://www.reddit.com/r/diabetes_t1/comments/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BRDRDT2-t1_impbcf4,I tried it for a little while. No side effects...,NaN,Metformin,Forums,Replies and Comments,No,No Media,https://www.reddit.com/r/diabetes_t1/comments/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1565738759353602048,i ran out of characters. youtu.be/RWgl2PDhQiM ...,NaN,NaN,Twitter,Replies and Comments,No,Link,http://twitter.com/bromid/statuses/15657387593...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17944607459251789,MY lunch! Ate at 10:30am \n1 unit NovoLog insu...,NaN,NaN,Instagram,Original,No,Image,https://www.instagram.com/p/Ch-gqL-Onqk/,https://scontent-iad3-1.cdninstagram.com/v/t51...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRDRDT2-t1_imq8h9m,This is also because like a soak in a hot tub ...,NaN,"No bath salts, bath oils, soaks?",Forums,Replies and Comments,No,No Media,https://www.reddit.com/r/diabetes/comments/x3e...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check the percentage of missing values in each column
percent_missing = round(df.isnull().sum() * 100 / len(df),2)
missing_value_df = pd.DataFrame({'percent_missing %': percent_missing})
missing_value_df

,percent_missing %
Post ID,0.00
Sound Bite Text,0.00
Ratings and Scores,100.00
Title,4.15
Source Type,0.00
...,...
LexisNexis Subject Group 1,100.00
LexisNexis Subject 1,100.00
LexisNexis Subject Group 2,100.00
LexisNexis Subject 2,100.00


In [ ]:
# Delete columns containing either 90% or more than 90% NaN Values
perc = 90.0
min_count =  int(((100-perc)/100)*df.shape[0] + 1)
df1 = df.dropna(axis=1,thresh=min_count)
df1.head()

,Post ID,Sound Bite Text,Title,Source Type,Post Type,Is Paid,Media Type,URL,Domain,Sentiment,...,Author Gender,Author URL,Author Name,Author Reddit Karma,Followers/Daily Unique Visitors/Subscribers,Positive Objects,Negative Objects,Richness,Reddit Score,Source Name
0,BRDRDT2-t1_imq98sr,My numbers are great now. Estimated a1c of 7%i...,Have you been denied a second/third pump? Feel...,Forums,Replies and Comments,No,No Media,https://www.reddit.com/r/diabetes_t1/comments/...,reddit.com,Neutrals,...,Unknown,https://www.reddit.com/user/itsgucciflipflopsbro,itsgucciflipflopsbro,4472,42768.0,number,NaN,0,0,r/diabetes_t1
1,BRDRDT2-t1_impbcf4,I tried it for a little while. No side effects...,Metformin,Forums,Replies and Comments,No,No Media,https://www.reddit.com/r/diabetes_t1/comments/...,reddit.com,Positives,...,Unknown,https://www.reddit.com/user/Unrealgecko,Unrealgecko,5502,42760.0,NaN,NaN,1,0,r/diabetes_t1
2,1565738759353602048,i ran out of characters. youtu.be/RWgl2PDhQiM ...,NaN,Twitter,Replies and Comments,No,Link,http://twitter.com/bromid/statuses/15657387593...,twitter.com,Positives,...,Unknown,http://twitter.com/bromid,dimo,-,1912.0,"dexcom g6, omnipod system",NaN,3,-,NaN
3,17944607459251789,MY lunch! Ate at 10:30am \n1 unit NovoLog insu...,NaN,Instagram,Original,No,Image,https://www.instagram.com/p/Ch-gqL-Onqk/,instagram.com,Neutrals,...,Unknown,NaN,NaN,-,NaN,NaN,NaN,0,-,NaN
4,BRDRDT2-t1_imq8h9m,This is also because like a soak in a hot tub ...,"No bath salts, bath oils, soaks?",Forums,Replies and Comments,No,No Media,https://www.reddit.com/r/diabetes/comments/x3e...,reddit.com,Neutrals,...,Unknown,https://www.reddit.com/user/rinkidinkidoo,rinkidinkidoo,2976,93772.0,NaN,NaN,2,0,r/diabetes


In [ ]:
# Check the percentage of missing values in each column
percent_missing = round(df1.isnull().sum() * 100 / len(df1),2)
missing_value_df = pd.DataFrame({'percent_missing %': percent_missing})
missing_value_df

,percent_missing %
Post ID,0.00
Sound Bite Text,0.00
Title,4.15
Source Type,0.00
Post Type,7.08
Is Paid,0.00
Media Type,0.00
URL,0.00
Domain,0.00
Sentiment,0.00


In [ ]:
df2 = df1.drop(['Positive Objects','Negative Objects'], axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37844 entries, 0 to 37843
Data columns (total 19 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Post ID                                      37844 non-null  object 
 1   Sound Bite Text                              37844 non-null  object 
 2   Title                                        36273 non-null  object 
 3   Source Type                                  37844 non-null  object 
 4   Post Type                                    35165 non-null  object 
 5   Is Paid                                      37844 non-null  object 
 6   Media Type                                   37844 non-null  object 
 7   URL                                          37844 non-null  object 
 8   Domain                                       37844 non-null  object 
 9   Sentiment                                    37844 non-null  object 
 10

In [ ]:
df2['Title'].fillna(value = 'Empty_Title', inplace = True)
df2['Sound Bite Text'] = df2['Sound Bite Text'] + " " + df2['Title']

In [ ]:
# remove links
df2["no links"] = df2["Sound Bite Text"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))
# remove punctuation
def remove_punctuation(test_str):
# Using filter() and lambda function to filter out punctuation characters
  result = ''.join(filter(lambda x: x.isalpha() or x.isdigit() or x.isspace(), test_str))
  return result
df2["no punc"] = df2["no links"].apply(lambda s: remove_punctuation(s))
# remove emojis
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
df2["clean_text"] = df2["no punc"].apply(lambda s: deEmojify(s))

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')  # load spaCy's built-in English models
texts = df2['clean_text'].to_numpy()
tokens = []
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
     
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()
for txt in texts:
    token = [word for word in tk.tokenize(txt.lower()) if word not in nlp.Defaults.stop_words]
    #token = [word for word in tk.tokenize(txt.lower())]
    #tokens.append(' '.join(token))
    tokens.append(token)

In [ ]:
df2["tokens"] = tokens
# df2["tokens"][3]

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
# Lemmatization
def text_lemmatize(text, position):
    lemmatizer = WordNetLemmatizer()
    words_lemmatize = []
    for i in text:
        words_lemmatize.append(WordNetLemmatizer().lemmatize(i, pos= position))
    #text_lemmatize = " ".join(words_lemmatize)
    return words_lemmatize

lemma_v = []
for i in range(0, len(df2)):
    lemma_v.append(text_lemmatize(df2["tokens"][i], "v"))
df2["lemma_v"] = lemma_v

lemma_n = []
for i in range(0, len(df2)):
    lemma_n.append(text_lemmatize(df2["lemma_v"][i], "n"))
df2["lemma_n"] = lemma_n

lemma_clean = []
for i in range(0, len(df2)):
    lemma_clean.append(text_lemmatize(df2["lemma_n"][i], "a"))
df2["lemma_clean"] = lemma_clean

df2[["Sound Bite Text", "lemma_clean"]]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,Sound Bite Text,lemma_clean
0,My numbers are great now. Estimated a1c of 7%i...,"[number, great, estimate, a1c, 7ish, doesnt, c..."
1,I tried it for a little while. No side effects...,"[try, little, effect, help, insulin, resistanc..."
2,i ran out of characters. youtu.be/RWgl2PDhQiM ...,"[run, character, youtuberwgl2pdhqim, ill, newl..."
3,MY lunch! Ate at 10:30am \n1 unit NovoLog insu...,"[lunch, eat, 1030am, 1, unit, novolog, insulin..."
4,This is also because like a soak in a hot tub ...,"[like, soak, hot, tub, potential, exist, low, ..."
...,...,...
37839,"Push for it, it's a game changer in my opinion...","[push, game, changer, opinion, course, come, i..."
37840,(My mom learned to inject my 10 year old self ...,"[mom, learn, inject, 10, year, old, self, oran..."
37841,I just spoke with a rep from Abbott. The rep t...,"[speak, rep, abbott, rep, tell, plan, update, ..."
37842,It’s been 2.5 weeks since I started and I’m fe...,"[25, week, start, im, feel, good, literal, yea..."


In [ ]:
# Regular expression for dexcom
text_dexcom_trimmed = []
for i in range(0, len(df2)):
    temp = [re.sub(r'dexcom.*', 'dexcom', text) for text in df2["lemma_clean"][i]]
    text_dexcom_trimmed.append(temp)

In [ ]:
# Regular expression for freestyle
text_free_trimmed = []
for i in range(0, len(df2)):
    temp = [re.sub(r'freestyle.*', 'freestyle', text) for text in text_dexcom_trimmed[i]]
    text_free_trimmed.append(temp)

In [ ]:
def getCategory(token):
    if 'dexcom' in token:
        if 'libre' in token or 'freestyle' in token:
            return 'both'
        else:
            return 'dexcom'
    elif 'libre' in token or 'freestyle' in token:
        return 'Freestyle Libre'
    elif 'dexcom' in token:
        return 'both' 
    else:
        return 'none'

In [ ]:
category = []
for token in text_libre_trimmed:
    category.append(getCategory(token))

#category = []
#for token in tokens:
#    category.append(getCategory(token))

df2["Category"] = category

NameError: ignored

In [ ]:
df2["Final_Text"] = text_libre_trimmed
# df2["Final_Text"][254]

In [ ]:
# Regular expression for libre
text_libre_trimmed = []
for i in range(0, len(df2)):
    temp = [re.sub(r'libre.*', 'libre', text) for text in text_free_trimmed[i]]
    text_libre_trimmed.append(temp)

In [ ]:
df3 = df2[["Sound Bite Text", "Title", "Source Type", "Post Type", "Is Paid", "Media Type", "Domain",
 "Sentiment", "Author Gender", "Category", "Final_Text"]]

In [ ]:
df_q1 = df3.copy()

In [ ]:
filter_list = ['expect', 'want', 'need','look', 'forward', 'require', 'anticipate', 'desire', 'prefer', 'hope', 'wish', 'envision', 'imagine', 'plan', 'consider', 'suggest']

In [ ]:
filtered_df = df_q1[df_q1['Final_Text'].apply(lambda x: any(item for item in x if item in filter_list))]

In [ ]:
filtered_df

In [ ]:
def clean_text(text):
    # Convert list to string
    text_str = ' '.join(text)
    # Remove non-alphanumeric characters except for spaces
    text_str = re.sub(r'[^a-zA-Z0-9\s]+', '', text_str)
    # Remove extra spaces
    text_str = re.sub(r'\s+', ' ', text_str)
    # Strip leading/trailing spaces
    text_str = text_str.strip()
    return text_str
  
filtered_df['Final_Text_spaced'] = filtered_df['Final_Text'].apply(clean_text)

In [ ]:
filtered_df

In [ ]:
    adjs = []
    nouns = []
    verbs = []

    for d in filtered_df['Final_Text_spaced']:
        for token in nlp(d):
            if token.pos_ == 'ADJ':
                adjs.append(token.text)
            elif token.pos_ == 'NOUN':
                nouns.append(token.text)
            elif token.pos_ == 'VERB':
                verbs.append(token.text)

    adj_counter = Counter(adjs)
    noun_counter = Counter(nouns)
    verb_counter = Counter(verbs) 

In [ ]:
adj_counter

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
levels = [1, 2, 3]
SHOW_NUMBER = 20
WORDCLOUD_NUMBER = 40
stopwords = STOPWORDS

In [ ]:
    wordcloud = WordCloud(background_color='white', max_font_size=50, max_words=100).generate_from_frequencies(adj_counter)
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
    wordcloud = WordCloud(background_color='white', max_font_size=50, max_words=100).generate_from_frequencies(noun_counter)
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
    wordcloud = WordCloud(background_color='white', max_font_size=50, max_words=100).generate_from_frequencies(verb_counter)
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
stop = ['freestyle', 'dexcom', 'blood', 'sugar', 'insulin', 'pump', 'diabetes', 'continuous', 'glucose', 'libre', 'x2', 'g6', 'medtronic', '770g', 'rdiabetes', 'tslim', 'cgm'] + filter_list

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, # any word shows up more than 95% of documents will be throw out. if integer 15, means more than 15 documents
                                min_df=2, # any word shows up less than 2 documents, it will be thrown out. it can be replaced with fraction like 0.05
                                stop_words=stop, # remove words in english stopwords
                                ngram_range = (3, 3)) 
tf = tf_vectorizer.fit_transform(filtered_df['Final_Text_spaced'])

In [ ]:
vocab = list(tf_vectorizer.get_feature_names())
counts = tf.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))

freq_distribution.most_common(50)

In [ ]:
confuse_list = ['confuse', 'unclear', 'misunderstand', 'misconception', 'unsure', 'clarification', 'inexperienced', 'inexperience', 'unfamiliar', 'complex', 'difficult', 'challange', 'challanging', 'lack', 'hard', 'gap', 'trouble', 'uncertain', 'hard', 'question', 'ask', 'trick', 'concern', 'complication', 'nightmare']

In [ ]:
confuse_df = df_q1[df_q1['Final_Text'].apply(lambda x: any(item for item in x if item in confuse_list))]
confuse_df

In [ ]:
confuse_df['Final_Text_spaced'] = confuse_df['Final_Text'].apply(clean_text)

In [ ]:
stop = ['blood', 'sugar', 'insulin', 'pump', 'diabetes', 'continuous', 'glucose', 'libre', 'dexcom', 'rdiabetes', 'amaze', 'live', 'im', 'cgm', 'carb', 'thread', 'tslim'] + confuse_list

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, # any word shows up more than 95% of documents will be throw out. if integer 15, means more than 15 documents
                                min_df=2, # any word shows up less than 2 documents, it will be thrown out. it can be replaced with fraction like 0.05
                                stop_words=stop, # remove words in english stopwords
                                ngram_range = (3, 3)) 
tf = tf_vectorizer.fit_transform(confuse_df['Final_Text_spaced'])

In [ ]:
vocab = list(tf_vectorizer.get_feature_names())
counts = tf.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))

freq_distribution.most_common(50)

In [ ]:
benefit_list = ['benefit']

In [ ]:
benefit_df = df_q1[df_q1['Final_Text'].apply(lambda x: any(item for item in x if item in benefit_list))]
benefit_df

In [ ]:
benefit_df['Final_Text_spaced'] = benefit_df['Final_Text'].apply(clean_text)

In [ ]:
stop = ['blood', 'continuous', 'cgm', 'freestyle', 'libre', 'dexcom', 'glucose', 'tslim', 'like', 'im', 'insulin', 'carb', '670g', '780g', '770g', 'medtronic', 'rdiabetes', 'continue', 'thread', 'g6', 'g7', 'europe', 'folk', 'diabetes']

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, # any word shows up more than 95% of documents will be throw out. if integer 15, means more than 15 documents
                                min_df=2, # any word shows up less than 2 documents, it will be thrown out. it can be replaced with fraction like 0.05
                                stop_words=stop, # remove words in english stopwords
                                ngram_range = (3, 3)) 
tf = tf_vectorizer.fit_transform(benefit_df['Final_Text_spaced'])

In [ ]:
vocab = list(tf_vectorizer.get_feature_names())
counts = tf.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))

freq_distribution.most_common(50)